In [1]:
# Confer : https://towardsdatascience.com/step-by-step-guide-to-using-pretrained-models-in-keras-c9097b647b29

In [2]:
# ImageMagic Install
!x=$(dpkg -l|grep imagemagick);[ ${#x} -eq 0 ]&&sudo apt install -y imagemagick

# Mount Google Drive
from google.colab import drive
isMount=!df |grep /content/drive
if len(isMount)<1: drive.mount('/content/drive')


In [3]:
%%bash
[ ! -f /content/drive/MyDrive/Lec_Capture/_data/flower_photos_300x200_small.tar  ]&&wget https://raw.githubusercontent.com/Finfra/AI_Vision/master/data/flower_photos_300x200_small.tar -O /content/drive/MyDrive/Lec_Capture/_data/flower_photos_300x200_small.tar 
tar xf /content/drive/MyDrive/Lec_Capture/_data/flower_photos_300x200_small.tar
find ./flower_photos_300x200_small|grep jpg|xargs -i -t convert {} -quiet -resize 224x224\! -quality 100 -colorspace RGB -type truecolor {} 2>> /tmp/log
find ./flower_photos_300x200_small|grep jpg|head -n 10|xargs -i -t identify {} 2>>/tmp/log

./flower_photos_300x200_small/daisy/14221848160_7f0a37c395.jpg JPEG 224x224 224x224+0+0 8-bit sRGB 95.8KB 0.000u 0:00.000
./flower_photos_300x200_small/daisy/19280272025_57de24e940_m.jpg JPEG 224x224 224x224+0+0 8-bit sRGB 45.9KB 0.000u 0:00.000
./flower_photos_300x200_small/daisy/483886997_27ee798327.jpg JPEG 224x224 224x224+0+0 8-bit sRGB 63.9KB 0.000u 0:00.000
./flower_photos_300x200_small/daisy/14600779226_7bbc288d40_m.jpg JPEG 224x224 224x224+0+0 8-bit sRGB 76.2KB 0.000u 0:00.009
./flower_photos_300x200_small/daisy/3780380240_ef9ec1b737_m.jpg JPEG 224x224 224x224+0+0 8-bit sRGB 53.6KB 0.000u 0:00.000
./flower_photos_300x200_small/daisy/2454280135_ac3aa75cdc_n.jpg JPEG 224x224 224x224+0+0 8-bit sRGB 76.8KB 0.000u 0:00.000
./flower_photos_300x200_small/daisy/154332674_453cea64f4.jpg JPEG 224x224 224x224+0+0 8-bit sRGB 70KB 0.000u 0:00.000
./flower_photos_300x200_small/daisy/3704305945_a80e60e2f6_m.jpg JPEG 224x224 224x224+0+0 8-bit sRGB 85.9KB 0.000u 0:00.000
./flower_photos_300x200

In [4]:
import tensorflow as tf
from tensorflow.python import keras
from tensorflow.keras import datasets, layers, models ,Input, Model

In [5]:
from __future__ import absolute_import, division, print_function, unicode_literals
from tensorflow.keras import datasets, layers, models, regularizers

import numpy as np
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join
from pylab import *
from numpy import *

def getFolder(thePath,isFile=True):
    return [f for f in listdir(thePath) if isFile == isfile(join(thePath, f)) ]


def getImagesAndLabels(tPath):
    labels=getFolder(tPath,False)
    encoder={w: i for i, w in enumerate(labels)}
    decoder={i: w  for i, w in enumerate(labels)}

    
    tImages,tLabels=None,None

    for label in labels:
        imgFolder=join(tPath,label)
        files= [f for f in listdir(imgFolder) if isfile(join(imgFolder, f))]
        for file in files:
            imageFile=join(imgFolder,file)
            img=np.array([imread(imageFile)])/255.
            if tImages is None:
                tImages, tLabels =img, np.array([encoder[label]],ndmin=1)
            else:
#                 print(tImages.shape,img.shape,imageFile)
                tImages = np.vstack( (tImages,img) )
                tLabels = np.append(tLabels,np.array([encoder[label]]) ,axis=0) 
        print(label,tImages.shape)                 
    return (tImages,tLabels,encoder,decoder)

    

from sklearn.model_selection import train_test_split
basePath='/content/flower_photos_300x200_small'
imgs,labels,encoder,decoder=getImagesAndLabels(basePath)

class_cnt=len(encoder.keys())

train_images,test_images,train_labels,test_labels=train_test_split(imgs,labels,test_size=0.3333, random_state=99)


imgShape=train_images[0].shape
img_width,img_hight,color_count=imgShape
train_images = train_images.reshape((-1,img_width,img_hight,color_count ))
test_images = test_images.reshape((-1, img_width,img_hight,color_count ))

print("Shape of Train_images = ",train_images.shape)

daisy (100, 224, 224, 3)
tulips (200, 224, 224, 3)
Shape of Train_images =  (133, 224, 224, 3)


In [6]:
from tensorflow.python.keras.applications.vgg16 import VGG16


model=keras.models.Sequential()
model.add(VGG16(weights='imagenet'))
# model.summary()

inp=Input(shape=(224,224,3))
out=VGG16(weights='imagenet')(inp)
model=Model(inputs=inp,outputs=out)
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
vgg16 (Functional)           (None, 1000)              138357544 
Total params: 138,357,544
Trainable params: 138,357,544
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=1,batch_size=10)

14/14 [==============================] - 1s 75ms/step - loss: 158.4393 - accuracy: 0.5113


In [8]:
score = model.evaluate(test_images, test_labels, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.8201956748962402
Test accuracy: 0.49253731966018677
